## Configuration of cairio.org

Right now the central cairio server is hosted on https://pairio.org:20443

This is a linode server

Inside a tmux session called cairio:

```
cd ~/src/spikeforest2/mountaintools/cairioserver
```

See .env file containing:

```
PORT=20443
CAIRIO_ADMIN_TOKEN=******
```

To run the server:

```
npm install .
node cairioserver.js
```

There are instructions in the home directory for setting up firewall and renewing the certs

In [1]:
%load_ext autoreload
%autoreload 2

# import cairio client and other packages
from mountaintools import client as mt
from mountaintools import CairioClient
from getpass import getpass
import os
from getpass import getpass
import json
from simplecrypt import encrypt, decrypt
import base64
from dotenv import load_dotenv

In [2]:
# Load secret stuff from the cairio.env file (only on the admin's computer)

load_dotenv(dotenv_path='cairio.env',verbose=True)
CAIRIO_ADMIN_TOKEN=os.environ['CAIRIO_ADMIN_TOKEN']
mt.setPairioToken('admin', CAIRIO_ADMIN_TOKEN)

In [3]:
# Let's set up the admin collection where we will store the user login information
# It will have the same token as the admin token

mt.addRemoteCollection(collection='admin', token=CAIRIO_ADMIN_TOKEN, admin_token=CAIRIO_ADMIN_TOKEN)

True

In [7]:
# Now let's add some users.

def set_user_config(user,config,password):
    key=dict(
        name='user_config',
        user=user
    )
    if password is None:
        password=getpass('Enter an encryption password for {}: '.format(user))
    val=base64.b64encode(encrypt(password,json.dumps(config))).decode()
    cc=CairioClient()
    cc.setPairioToken(collection='admin',token=CAIRIO_ADMIN_TOKEN)
    cc.setValue(
        key=key,
        value=val,
        collection='admin'
    )

# user=magland, jjun
config_sf=dict(
    kbucket_shares=[
        dict(hub='kbucket.flatironinstitute.org',name='sf_raw',node_id='15734439d8cf'),
        dict(hub='kbucket.flatironinstitute.org',name='spikeforest1',node_id='7317cea8265b',upload_token=os.environ['SPIKEFOREST1_UPLOAD_TOKEN']),
        dict(hub='kbucket.flatironinstitute.org',name='spikeforest2',node_id='69432e9201d0',upload_token=os.environ['SPIKEFOREST2_UPLOAD_TOKEN']),
        dict(hub='kbucket.flatironinstitute.org',name='spikeforest3',node_id='f044bb130923',upload_token=os.environ['SPIKEFOREST3_UPLOAD_TOKEN'])
    ],
    cairio_collections=[
        dict(database='default',name='spikeforest',token=os.environ['SPIKEFOREST_TOKEN'])
    ],
    pairio_tokens=dict(
        spikeforest=os.environ['SPIKEFOREST_TOKEN']
    ),
    kachery_upload_tokens={
        'public':os.environ['KACHERY_PUBLIC_TOKEN'],
        'kbucket':os.environ['KACHERY_KBUCKET_TOKEN'],
        'spikeforest.public':os.environ['KACHERY_PUBLIC_TOKEN'],
        'spikeforest.kbucket':os.environ['KACHERY_KBUCKET_TOKEN']
    }
)
# user=morley
config_morley=dict(
    kbucket_shares=[
        dict(hub='kbucket.flatironinstitute.org',name='morleyshare',node_id='b906f94785d5',upload_token=os.environ['MORLEYSHARE_UPLOAD_TOKEN'])
    ],
    cairio_collections=[
        dict(database='default',name='morley',token=os.environ['MORLEY_TOKEN'])
    ],
    pairio_tokens=dict(
        morley=os.environ['MORLEY_TOKEN']
    ),
    kachery_upload_tokens={
        'public':os.environ['KACHERY_PUBLIC_TOKEN'],
        'spikeforest.public':os.environ['KACHERY_PUBLIC_TOKEN']
    }
)
set_user_config('magland',config_sf,password=os.getenv('MAGLAND_PASSWORD'))
set_user_config('jjun',config_sf,password=os.getenv('JJUN_PASSWORD'))
set_user_config('morley',config_morley,password=os.getenv('MORLEY_PASSWORD'))

In [8]:
# Now this is how we would log in as magland and configure the cairio client:

mt.login(user='magland', ask_password=True)
mt.configDownloadFrom(['spikeforest.kbucket'])

Mountain password for magland:  ·····


Logging in as magland...
Logged in as magland


In [7]:
# Let's set the token for the remote collections

mt.addRemoteCollection(collection='spikeforest', token=os.environ['SPIKEFOREST_TOKEN'], admin_token=CAIRIO_ADMIN_TOKEN)
mt.addRemoteCollection(collection='morley', token=os.environ['MORLEY_TOKEN'], admin_token=CAIRIO_ADMIN_TOKEN)

True

In [9]:
# test morley login
mt.login(user='morley',ask_password=True)
mt.setValue(key='morleyshare',value='b906f94785d5')

Mountain password for morley:  ········


Logging in as morley...
Logged in as morley


True

In [10]:
mt.saveObject(key='testkey',object=dict(test='object'))

'sha1://81b50a6e50e4553389e6be429fb28841e0249f75/object.json'

In [13]:
###### Let's do a test
mt.login(user='magland',ask_password=True)
mt.configDownloadFrom('spikeforest.spikeforest2')

mt.saveObject(key='test1',object=dict(test=1))
print(mt.loadObject(key='test1'))
mt.saveObject(key='test1',object=dict(test=2))
print(mt.loadObject(key='test1'))

Mountain password for magland:  ·····


Logging in as magland...
Logged in as magland
{'test': 1}
{'test': 2}


In [14]:
# Let's make aliases so we don't need to remember the share_id for the kbucket shares
mt.setValue(key='spikeforest2',value='69432e9201d0')
mt.setValue(key='spikeforest3',value='f044bb130923')

True

In [15]:
# Now we can do this!
mt.configDownloadFrom('spikeforest.spikeforest2')

In [6]:
# Set up the kachery aliases
mt.setValue(key='kbucket', value='http://pairio.org:12001', collection='spikeforest')
mt.setValue(key='public', value='http://45.79.176.243:8080', collection='spikeforest')

True